# Multi-Class Classification using BERT

Classifying the sentences from IRs (Instruction Reports) into different labels of "Human Errors" i.e.: <Br>
T: Team Cognition <br>
P: Procedural <br>
O: Organizational <br>
D: Design <br>
H: Human <br>
<br>
For Training Data, we have sentences labelled by the coders - Sally, Frenard and Trixy


## 1. Configuring GPU and Libraries

In [1]:
# verify GPU availability
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Gen RAM Free: 12.5 GB  | Proc size: 498.0 MB
GPU RAM Free: 14968MB | Used: 111MB | Util   1% | Total 15079MB


In [3]:
!ps ax | grep python

     23 ?        Sl     0:16 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
   5003 ?        Ssl    0:02 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-390d47d1-4673-4303-a7ba-f5c0e19e8e10.json
   5072 ?        S      0:00 /bin/bash -c ps ax | grep python
   5074 ?        S      0:00 grep python


In [4]:
!pip install pytorch-pretrained-bert pytorch-nlp

In [5]:
# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import string
import os
import glob
% matplotlib inline

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

Using TensorFlow backend.


'Tesla T4'

In [0]:
torch.cuda.empty_cache()

In [8]:
from google.colab import drive
drive.mount("/GD")

Drive already mounted at /GD; to attempt to forcibly remount, call drive.mount("/GD", force_remount=True).


In [7]:
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from sklearn.model_selection import train_test_split
import os

print("tensorflow version : ", tf.__version__)
print("tensorflow_hub version : ", hub.__version__)

tensorflow version :  1.15.0
tensorflow_hub version :  0.7.0


In [10]:

#Installing BERT module
!pip install bert-tensorflow

In [8]:

#Importing BERT modules
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [9]:
OUTPUT_DIR = '/GD/My Drive/Colab Notebooks/DS_Lab/Models/'

#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    pass

tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: /GD/My Drive/Colab Notebooks/DS_Lab/Models/ *****


## 2. Data Preprocessing

In [0]:
df = []

In [11]:
files = [f for f in glob.glob("*.csv")]
df_raw = pd.DataFrame()

for file in files:
 
  df = pd.read_csv(file)
  df['text'] = df['text'].apply(str)
  df_raw = pd.concat([df_raw , df], ignore_index=True)

df = df_raw.dropna()
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,end_pos,file,label_Frenard,label_SALLY,line,start_pos,text
0,14,ML14087A338.txt,U,U,0,0,28-Mar-14
1,422,ML14087A338.txt,U,U,2,0,"Jeremy Browning, Site Vice President Arkansas ..."
2,559,ML14087A338.txt,U,U,2,422,The enclosed report documents the inspection r...
3,701,ML14087A338.txt,U,U,2,559,The team discussed the revised inspection resu...
4,799,ML14087A338.txt,U,U,2,701,The team reviewed selected procedures and reco...


In [0]:
TRAINING_LABEL = ['label_Frenard', 'label_SALLY']

def get_similar(row):
    if len(TRAINING_LABEL)==2:
        if row[TRAINING_LABEL[0]] == row[TRAINING_LABEL[1]] and row[TRAINING_LABEL[0]] != 'U':
            return row[TRAINING_LABEL[0]]
        else:
            return np.NaN
    if len(TRAINING_LABEL)==3:
        if row[TRAINING_LABEL[0]] == row[TRAINING_LABEL[1]] == row[TRAINING_LABEL[2]] and row[TRAINING_LABEL[0]] != 'U':
            return row[TRAINING_LABEL[0]]
        else:
            return np.NaN


def get_different(row):
    if len(TRAINING_LABEL) == 2:
        if row[TRAINING_LABEL[0]] == 'U' or row[TRAINING_LABEL[0]] == row[TRAINING_LABEL[1]]:
            return np.NaN
        else:
            return row[TRAINING_LABEL[0]]
    if len(TRAINING_LABEL) == 3:
        if row[TRAINING_LABEL[0]] == 'U' or row[TRAINING_LABEL[0]] == row[TRAINING_LABEL[1]] == row[TRAINING_LABEL[2]]:
            return np.NaN
        else:
            return row[TRAINING_LABEL[0]]

In [13]:
def class_label_handler(class_label):
  if 'U' in class_label:
    return np.NaN
  else:
    return class_label.split(',')[0]

for label in TRAINING_LABEL:  
  df[label] = df[label].apply(class_label_handler)

if len(TRAINING_LABEL) > 1:
  df['Y'] = df.apply(get_different, axis=1)

for label in TRAINING_LABEL:
  df = df.drop(label, axis=1)
  
df = df.dropna()

label_idx = {'T':0, 'P': 1, 'O': 2, 'D': 3, 'H': 4}
def changing_labels_to_integers(class_label):
  return label_idx[class_label]

df['Y']=df['Y'].apply(changing_labels_to_integers)
len(df['Y'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


133

In [0]:
# Splitting Training and Test
TRAIN_TEST_SPLIT = 0.8
msk = np.random.rand(len(df)) < TRAIN_TEST_SPLIT
df_train = df[msk]
df_test = df[~msk]

In [0]:
from sklearn.model_selection import train_test_split

train, val =  train_test_split(df_train, test_size = 0.2, random_state = 100)

In [16]:
print("Training Set Shape :", train.shape)
print("Validation Set Shape :", val.shape)
print("Test Set Shape :", df_test.shape)

Training Set Shape : (87, 6)
Validation Set Shape : (22, 6)
Test Set Shape : (24, 6)


In [0]:
DATA_COLUMN = 'text'
LABEL_COLUMN = 'Y'
# The list containing all the classes (train['SECTION'].unique())
label_list = [0, 1, 2, 3, 4]

BERT model accept only a specific type of input and the datasets are usually structured to have have the following four features:

<ul> guid : A unique id that represents an observation.</ul>
<ul> text_a : The text we need to classify into given categories </ul>
<ul> text_b: It is used when we're training a model to understand the relationship between sentences and it does not apply for classification problems. </ul>
 <ul>label: It consists of the labels or classes or categories that a given text belongs to. </ul>

In our dataset we have text_a and label. The following code block will create objects for each of the above mentioned features for all the records in our dataset using the InputExample class provided in the BERT library.

In [0]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = val.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [19]:
train_InputExamples

3176    <bert.run_classifier.InputExample object at 0x...
310     <bert.run_classifier.InputExample object at 0x...
3537    <bert.run_classifier.InputExample object at 0x...
3896    <bert.run_classifier.InputExample object at 0x...
3891    <bert.run_classifier.InputExample object at 0x...
                              ...                        
3889    <bert.run_classifier.InputExample object at 0x...
3987    <bert.run_classifier.InputExample object at 0x...
3519    <bert.run_classifier.InputExample object at 0x...
624     <bert.run_classifier.InputExample object at 0x...
285     <bert.run_classifier.InputExample object at 0x...
Length: 87, dtype: object

In [20]:
print("Row 0 - guid of training set : ", train_InputExamples.iloc[0].guid)
print("\n__________\nRow 0 - text_a of training set : ", train_InputExamples.iloc[0].text_a)
print("\n__________\nRow 0 - text_b of training set : ", train_InputExamples.iloc[0].text_b)
print("\n__________\nRow 0 - label of training set : ", train_InputExamples.iloc[0].label)

Row 0 - guid of training set :  None

__________
Row 0 - text_a of training set :  Specifically, 2 of 8 crews (25 percent) failed the simulator scenario portion of the operating test; and 11 of 46 licensed operators (23 percent) either failed the scenario or failed the job performance measure portions of the operating tests.

__________
Row 0 - text_b of training set :  None

__________
Row 0 - label of training set :  2


The following code block loads the pre-trained BERT model and initializers a tokenizer object for tokenizing the texts.

In [21]:
# This is a path to an uncased (all lowercase) version of BERT

#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [22]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

# Convert our train and validation features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 87


INFO:tensorflow:Writing example 0 of 87


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 3 (id = 3)


INFO:tensorflow:label: 3 (id = 3)


In [23]:
#Example on first observation in the training set
print("Sentence : ", train_InputExamples.iloc[0].text_a)
print("-"*30)
print("Tokens : ", tokenizer.tokenize(train_InputExamples.iloc[0].text_a))
print("-"*30)
print("Input IDs : ", train_features[0].input_ids)
print("-"*30)
print("Input Masks : ", train_features[0].input_mask)
print("-"*30)
print("Segment IDs : ", train_features[0].segment_ids)

Sentence :  Specifically, 2 of 8 crews (25 percent) failed the simulator scenario portion of the operating test; and 11 of 46 licensed operators (23 percent) either failed the scenario or failed the job performance measure portions of the operating tests.
------------------------------
Tokens :  ['specifically', ',', '2', 'of', '8', 'crews', '(', '25', 'percent', ')', 'failed', 'the', 'simulator', 'scenario', 'portion', 'of', 'the', 'operating', 'test', ';', 'and', '11', 'of', '46', 'licensed', 'operators', '(', '23', 'percent', ')', 'either', 'failed', 'the', 'scenario', 'or', 'failed', 'the', 'job', 'performance', 'measure', 'portions', 'of', 'the', 'operating', 'tests', '.']
------------------------------
Input IDs :  [101, 4919, 1010, 1016, 1997, 1022, 10604, 1006, 2423, 3867, 1007, 3478, 1996, 25837, 11967, 4664, 1997, 1996, 4082, 3231, 1025, 1998, 2340, 1997, 4805, 7000, 9224, 1006, 2603, 3867, 1007, 2593, 3478, 1996, 11967, 2030, 3478, 1996, 3105, 2836, 5468, 8810, 1997, 1996, 4

## 3. Creating a Multi-Class Classifier Model

In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  
  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [0]:
#A function that adapts our model to work for training, evaluation, and prediction.

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        
        return {
            "eval_accuracy": accuracy,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
            }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 40.0
# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 300
SAVE_SUMMARY_STEPS = 100

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [27]:
#Initializing the model and the estimator
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '/GD/My Drive/Colab Notebooks/DS_Lab/Models/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0022f1c6a0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/GD/My Drive/Colab Notebooks/DS_Lab/Models/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0022f1c6a0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

# Create an input function for validating. drop_remainder = True for using TPUs.
val_input_fn = run_classifier.input_fn_builder(
    features=val_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

## 4. Training & Evaluating

In [29]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 108 into /GD/My Drive/Colab Notebooks/DS_Lab/Models/model.ckpt.


INFO:tensorflow:Saving checkpoints for 108 into /GD/My Drive/Colab Notebooks/DS_Lab/Models/model.ckpt.


INFO:tensorflow:Loss for final step: 0.03808254.


INFO:tensorflow:Loss for final step: 0.03808254.


Training took time  0:02:44.561071


In [30]:
#Evaluating the model with Validation set
val_acc = estimator.evaluate(input_fn=val_input_fn, steps=None)['eval_accuracy']

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Finished evaluation at 2020-02-18-01:04:12


INFO:tensorflow:Saving dict for global step 108: eval_accuracy = 0.36363637, false_negatives = 3.0, false_positives = 3.0, global_step = 108, loss = 3.2996569, true_negatives = 2.0, true_positives = 14.0


INFO:tensorflow:Saving dict for global step 108: eval_accuracy = 0.36363637, false_negatives = 3.0, false_positives = 3.0, global_step = 108, loss = 3.2996569, true_negatives = 2.0, true_positives = 14.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 108: /GD/My Drive/Colab Notebooks/DS_Lab/Models/model.ckpt-108


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 108: /GD/My Drive/Colab Notebooks/DS_Lab/Models/model.ckpt-108


## 5. Predicting For Test Set

In [0]:
"""{'T':0, 'P': 1, 'O': 2, 'D': 3, 'H': 4} 3"""

# A method to get predictions
def getPrediction(df_test):

  in_sentences = list(df_test['text'])
  #A list to map the actual labels to the predictions
  labels = ['T', 'P', 'O', 'D', 'H']

  #Transforming the test data into BERT accepted form
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] 
  
  #Creating input features for Test data
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

  #Predicting the classes 
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  pred = [labels[prediction['labels']] for prediction in predictions]
  df_test['Y'] = [labels[Y] for Y in list(df_test['Y'])]
  df_test['Predicted_Values'] = pred

  total_matching = sum(df_test['Y'] == pred)
  total = len(pred)
  acc = total_matching/total
  print('accuracy = ',acc)
  return acc

In [32]:
test_evals = getPrediction(df_test)
test_acc = test_evals

INFO:tensorflow:Writing example 0 of 24


INFO:tensorflow:Writing example 0 of 24


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


accuracy =  0.5416666666666666


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [33]:
df_test.to_csv('/GD/My Drive/Colab Notebooks/DS_Lab/Results/Frenard_Sally_DisAgree.csv', index = False)
df_test.head()

,end_pos,file,line,start_pos,text,Y,Predicted_Values
48,2997,ML14087A338.txt,40,2883,The team determined that this finding had a hu...,H,H
187,1791,ML14087A338.txt,82,1683,The licensee could not replace the bolts becau...,D,O
190,2485,ML14087A338.txt,82,2258,The team determined that the licensee had not ...,P,O
213,3068,ML14087A338.txt,84,2751,The team recommended that the system engineer ...,P,O
278,1413,ML14087A338.txt,96,1221,"However, during discussions the team determine...",P,P


In [0]:
from sklearn.metrics import confusion_matrix

pred_labels = list(set(df_test['Y']))

conf_mat = confusion_matrix(df_test['Y'], df_test['Predicted_Values'], pred_labels)


In [35]:
final_result = pd.DataFrame(columns = ['Coder','Train_Accuracy','Val_Accuracy','Test_Accuracy','#TestData','#AllData'])
list = []
list.append('Frenard_Sally_DisAgree')
final_result['Coder'] = list
list = []
list.append(val_acc)
final_result['Val_Accuracy'] = list
list = []
list.append(test_acc)
final_result['Test_Accuracy'] = list
list = []
list.append(len(df_test['text']))
final_result['#TestData'] = list
list = []
list.append(len(df['text']))
final_result['#AllData'] = list
list = []
list.append(pred_labels)
final_result['Classes'] = list
list = []
list.append(conf_mat)
final_result['Confusion Matrix'] = list
final_result

,Coder,Train_Accuracy,Val_Accuracy,Test_Accuracy,#TestData,#AllData,Classes,Confusion Matrix
0,Frenard_Sally_DisAgree,NaN,0.363636,0.541667,24,133,"[D, H, P, O]","[[1, 0, 0, 2], [0, 1, 1, 1], [0, 0, 2, 5], [0,..."


In [0]:
if not tf.io.gfile.exists("/GD/My Drive/Colab Notebooks/DS_Lab/Results/Final_Results.csv"):
  final_result.to_csv("/GD/My Drive/Colab Notebooks/DS_Lab/Results/Final_Results.csv", header=True)
else:
  final_result.to_csv("/GD/My Drive/Colab Notebooks/DS_Lab/Results/Final_Results.csv", mode='a', header=False)